Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 5.02 ms, sys: 700 µs, total: 5.72 ms
Wall time: 12.2 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.98/10244/1 Dashboard: http://10.20.0.98:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 257 ms, sys: 28.4 ms, total: 286 ms
Wall time: 2.22 s


[1.9357699039812921,
 1.076226241165258,
 1.7808762568475134,
 1.3731836257140162,
 1.0314434507491599,
 1.6028769230025013,
 1.6815503566328396,
 1.357502471146085,
 1.737168980513709,
 1.9006009770147385,
 1.2199679447484606,
 1.6038617142185354,
 1.3724812388281213,
 1.0921638600553603,
 1.862956480202733,
 1.7616078848837131,
 1.2121144324367306,
 1.8476683555613849,
 1.4676939997593386,
 1.303148311528341,
 1.9404067517810288,
 1.6690821954516664,
 1.2787079519451168,
 1.7762827073169265,
 1.36383820594305,
 1.0128115508202788,
 1.8801728090302852,
 1.6729604420867026,
 1.9153206285469273,
 1.705622737809696,
 1.107795077222336,
 1.1535123814178276,
 1.4593224465047798,
 1.9173947585928799,
 1.5564310656989306,
 1.1635279509118415,
 1.7147156318846513,
 1.4245687769655757,
 1.9394386684619773,
 1.4946340821409627,
 1.8169009472020001,
 1.4456644388741888,
 1.1305420563423818,
 1.2457779777202427,
 1.7953786846580586,
 1.5723576964014137,
 1.8008957528607452,
 1.744988840501807,
 1

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.773915703393401,
 1.4230082745840718,
 1.5608987799010654,
 1.0650468241947415,
 1.8591456241912174,
 1.664545294159367,
 1.7734689838811215,
 1.3051321729697987,
 1.6069125581576404,
 1.258435002276897,
 1.441355898117703,
 1.725486963774136,
 1.155150043750028,
 1.4629791977216864,
 1.8505945856851524,
 1.4483965296477033,
 1.0698807316723467,
 1.848732723582913,
 1.1205891047363719,
 1.1319190419459106,
 1.2836041825585078,
 1.460690026345341,
 1.8811845907046014,
 1.4909102406911048,
 1.0569405176900775,
 1.9022898980032197,
 1.9999638386748106,
 1.240183077894783,
 1.6784898522898057,
 1.5630246277999473,
 1.671778745668674,
 1.8402536007925954,
 1.9870925207293335,
 1.3330596893797217,
 1.1529940771971239,
 1.82871993666657,
 1.1218026840375042,
 1.9518107113377887,
 1.3075392092815856,
 1.4744184580310251,
 1.96035329023809,
 1.4980963186029925,
 1.6693522199336024,
 1.675482681134139,
 1.2282583937177285,
 1.0402762363054867,
 1.0182855690324708,
 1.2709696423782564,
 1.9302

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 199 ms, sys: 32.4 ms, total: 232 ms
Wall time: 2.22 s
